In [14]:
import argparse
import os
import logging as log
import torch
import matplotlib
from dataloader.asimow_dataloader import DataSplitId, ASIMoWDataModule
from dataloader.latentspace_dataloader import LatentPredDataModule
from dataloader.utils import get_val_test_ids, get_experiment_ids, get_val_test_experiments
from model.vq_vae import VectorQuantizedVAE
from model.vq_vae_patch_embedd import VQVAEPatch
from lightning.pytorch.loggers.wandb import WandbLogger
from lightning.pytorch.loggers.csv_logs import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning import Trainer
from model.mlp import MLP
from model.gru import GRU
from model.classification_model import ClassificationLightningModule
import numpy as np
# import tqdm.auto as tqdm
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [15]:
get_experiment_ids(1) + get_experiment_ids(2)

((1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 30),
 (1, 31),
 (1, 32),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 15),
 (2, 16),
 (2, 20),
 (2, 21))

In [16]:
def print_training_input_shape(data_module):
    data_module.setup(stage="fit")
    val_loader = data_module.val_dataloader()
    batch = next(iter(val_loader))
    for i in range(len(batch)):
        log.info(f"Input {i} shape: {batch[i].shape}")

In [17]:
def classify_latent_space(latent_model: VectorQuantizedVAE | VQVAEPatch, logger: CSVLogger | WandbLogger, val_ids: list[DataSplitId], 
                          test_ids: list[DataSplitId], n_cycles: int, model_name: str, dataset: str,
                          classification_model: str, learning_rate: float, clipping_value: float):

    # Initialize a data module for latent space prediction
    data_module = LatentPredDataModule(latent_space_model=latent_model, model_name=f"{model_name}", val_data_ids=val_ids, test_data_ids=test_ids,
                                       n_cycles=n_cycles, task='classification', batch_size=128, model_id=f"{model_name}-{dataset}")
    # Print the shape of training data for verification
    print("_______________________")
    print("Shape:")
    print_training_input_shape(data_module)
    print("_______________________")
    # Calculate sequence length and input dimension for the classification model
    seq_len = n_cycles
    input_dim = int(latent_model.embedding_dim * latent_model.enc_out_len)

    # Select the classification model (MLP or GRU) based on the provided argument
    Model: type[MLP] | type[GRU]
    if classification_model == "MLP":
        Model = MLP
    elif classification_model == "GRU":
        Model = GRU
    else:
        raise ValueError(f"Invalid classification model name: {classification_model}")

    # Initialize the classification model with specified parameters
    model = Model(input_size=seq_len, in_dim=input_dim, hidden_sizes=128, dropout_p=0.1,
                  n_hidden_layers=4, output_size=2, learning_rate=learning_rate)

    # Set up checkpointing and early stopping based on F1 score
    model_checkpoint_name = f"VQ-VAE-{classification_model}-{dataset}-best"
    checkpoint_callback = ModelCheckpoint(
        dirpath=f"model_checkpoints/VQ-VAE-{classification_model}/", monitor=f"val/f1_score", mode="max", filename=model_checkpoint_name)
    early_stop_callback = EarlyStopping(
        monitor=f"val/f1_score", min_delta=0.0001, patience=10, verbose=False, mode="max")

    # Initialize the PyTorch Lightning trainer with specified configurations
    trainer = Trainer(
        max_epochs=1,
        logger=logger,
        callbacks=[checkpoint_callback, early_stop_callback],
        devices=1,
        num_nodes=1,
        gradient_clip_val=clipping_value,
        check_val_every_n_epoch=1
    )

    # Train the model using the data module
    trainer.fit(
        model=model,
        datamodule=data_module,
    )

    # Log the best F1 score and accuracy obtained during validation
    best_score = model.hyper_search_value
    best_acc_score = model.val_acc_score
    print(f"best score: {best_score}")
    print("------ Testing ------")

    # Reinitialize the trainer for testing
    trainer = Trainer(
        devices=1,
        num_nodes=1,
        logger=logger,
    )

    # Test the model using the test data
    trainer.test(model=model, dataloaders=data_module)
    test_f1_score = model.test_f1_score
    test_acc = model.test_acc_score

    # Log test metrics (F1 score and accuracy)
    logdict = {"val/mean_f1_score": best_score, 
               "val/mean_acc": best_acc_score,
               "test/mean_f1_score": test_f1_score,
               "test/mean_acc": test_acc}
    
    # Log metrics using the appropriate logger (CSV or Wandb)
    if isinstance(logger, CSVLogger):
        logger.experiment.log_metrics(logdict)
    else: 
        logger.experiment.log(logdict)
        logger.experiment.finish()

    # Clean up the data folder used by the data module
    log.info("Cleaning up latent dataloader folder")
    data_folder = data_module.latent_dataloader.dataset_path
    os.system(f"rm -rf {data_folder}")

In [18]:
def get_losses(dataloader: ASIMoWDataModule, model: VQVAEPatch) -> (np.array, np.array, np.array):
    # device = "cuda" if torch.cuda.is_available() else "cpu"
    device = "cpu"
    model.to(device)
    recon_loss = []
    embedding_losses=[]
    perplexities = []
    model.eval()
    for batch in dataloader:
        input_data = batch[0].to(device)
        with torch.inference_mode():
            embedding_loss, x_hat, perplexity = model(input_data)
            recon_loss.append(np.mean((input_data.cpu().numpy() - x_hat.cpu().numpy())**2))
            embedding_losses.append(embedding_loss.cpu().numpy())
            perplexities.append(perplexity.cpu().numpy())
    return np.array(embedding_losses), np.array(recon_loss), np.array(perplexities)

In [19]:
def print_loss_stats(embed_loss: np.array, recon_loss: np.array, perplexity:np.array) -> None:
    print("Embedding loss:")
    print(f"Max: {np.max(embed_loss)}")
    print(f"Min: {np.min(embed_loss)}")
    print(f"Mean: {np.mean(embed_loss)}")
    print(f"Variance: {np.var(embed_loss)}")
    print("-------------------------------------------------------------")
    print("Reconstruction loss:")
    print(f"Max: {np.max(recon_loss)}")
    print(f"Min: {np.min(recon_loss)}")
    print(f"Mean: {np.mean(recon_loss)}")
    print(f"Variance: {np.var(recon_loss)}")
    print("-------------------------------------------------------------")
    corr_coeff = np.corrcoef(embed_loss, recon_loss)[0, 1]
    print("Pearson Correlation Coefficient: ", corr_coeff)
    print("-------------------------------------------------------------")
    # print("Perplexity: ")
    # print(f"Max: {np.max(perplexity)}")
    # print(f"Min: {np.min(perplexity)}")
    # print(f"Mean: {np.mean(perplexity)}")
    # print(f"Variance: {np.var(perplexity)}")
    # print("-------------------------------------------------------------\n")

In [20]:
batch_size = 64
model_path = "model_checkpoints/VQ-VAE-Patch/VQ-VAE-Patch-asimow-10-epochs.ckpt"

In [21]:
dataset_dict = get_val_test_ids()
val_ids = dataset_dict["val_ids"]
test_ids = dataset_dict['test_ids']

dataset_dict

# Log the validation and test IDs
# print(val_ids)
# print(test_ids)

{'test_ids': ((3, 32),
  (3, 18),
  (1, 27),
  (3, 19),
  (3, 17),
  (2, 21),
  (1, 20),
  (1, 11)),
 'val_ids': ((3, 3),
  (2, 10),
  (1, 24),
  (3, 24),
  (1, 32),
  (2, 1),
  (1, 10),
  (1, 16))}

In [22]:
val_ids = [DataSplitId(experiment=e, welding_run=w) for e, w in val_ids]
test_ids = [DataSplitId(experiment=e, welding_run=w) for e, w in test_ids]
data_module = ASIMoWDataModule(task="reconstruction", 
                               batch_size=batch_size, 
                               n_cycles=1, 
                               val_data_ids=val_ids, 
                               test_data_ids=test_ids,
                               shuffle_val_test=False)
# input_dim = 2


In [23]:
data_module.setup(stage="")

In [24]:
test_dl = data_module.test_dataloader()
print(len(test_dl))
for batch in test_dl:
    print(batch)


356
tensor([[[-0.4462, -0.6315],
         [-0.3031, -0.4769],
         [-0.0973, -0.2463],
         ...,
         [-0.6593, -0.6575],
         [-0.6593, -0.6575],
         [-0.6593, -0.6575]],

        [[-0.6085, -0.6424],
         [-0.5088, -0.5324],
         [-0.2929, -0.3103],
         ...,
         [-0.5921, -0.6534],
         [-0.5921, -0.6534],
         [-0.5921, -0.6534]],

        [[-0.6208, -0.6515],
         [-0.4656, -0.5776],
         [-0.2149, -0.3695],
         ...,
         [-0.7708, -0.6590],
         [-0.7708, -0.6590],
         [-0.7708, -0.6590]],

        ...,

        [[-0.8246, -0.6473],
         [-0.6568, -0.5339],
         [-0.4382, -0.3216],
         ...,
         [-0.9359, -0.6616],
         [-0.9359, -0.6616],
         [-0.9359, -0.6616]],

        [[-0.9278, -0.6526],
         [-0.7639, -0.5391],
         [-0.5301, -0.3318],
         ...,
         [-0.7888, -0.6428],
         [-0.7888, -0.6428],
         [-0.7888, -0.6428]],

        [[-0.7801, -0.6447],
   

In [25]:
data_module.setup(stage="test")
# len(data_module.train_dataloader())
# train_dl = data_module.train_dataloader()
# val_dl = data_module.val_dataloader()
test_dl = data_module.test_dataloader()
data_module.asimow_dataloader.load_dataset()

# test_dl = data_module.test_dataloader()

,t_wn,experiment,welding_run,labels,V,I
0,0,1,2,-1,"[14.72135416431125, 15.63385416416525, 16.2171...","[31.575520837375, 61.58854167455, 103.19010417..."
1,1,1,2,-1,"[13.9447916644355, 14.31302083104325, 15.04374...","[25.39062500325, 37.95572917152501, 71.4192708..."
2,2,1,2,-1,"[1.329166666454, 1.7567708330522502, 2.2322916...","[90.039062511525, 118.033854181775, 158.854166..."
3,3,1,2,-1,"[14.66666666432, 15.091145830918752, 16.007291...","[28.841145837025003, 47.98177083947501, 84.635..."
4,4,1,2,-1,"[3.3239583328015003, 3.56614583276275, 4.11458...","[283.13802086957503, 294.66145837105, 329.9479..."
...,...,...,...,...,...,...
209180,209180,3,32,-1,"[18.772078, 19.866151825, 20.660836495, 21.478...","[42.103725375, 69.14150537500001, 109.42939037..."
209181,209181,3,32,-1,"[18.8826703, 20.008341925, 20.89071049, 21.613...","[37.126225375, 56.329420375, 92.834405375, 135..."
209182,209182,3,32,-1,"[18.415812805, 19.02802018, 19.934087095, 20.9...","[38.131680375, 51.27228037500001, 87.737445375..."
209183,209183,3,32,-1,"[18.793406515, 19.83376408, 20.589741445, 21.3...","[41.705525375, 68.743305375, 109.359705375, 15..."


In [26]:
for i in [1, 2, 3]:
    for e, w in get_experiment_ids(i):
            id = [DataSplitId(experiment=e, welding_run=w)]
            module=ASIMoWDataModule(task="reconstruction", 
                                                    batch_size=batch_size, 
                                                    n_cycles=1, 
                                                    val_data_ids=val_ids, 
                                                    test_data_ids=id,
                                                    shuffle_val_test=False)
            module.setup(stage="test")
            dataloader = module.test_dataloader()
            with open(f"dataloader_pickles/dataloader_experiment_{e}_run_{w}.pkl", "wb") as file:
                  pickle.dump(dataloader, file)

In [27]:
with open("dataloader_pickles/dataloader_experiment_1_run_10.pkl", 'rb') as file:
    dataloader = pickle.load(file)
for batch in dataloader:
    print(batch)
    break

tensor([[[-1.0637, -0.6652],
         [-1.0912, -0.5384],
         [-0.9468, -0.3270],
         ...,
         [-4.1602, -0.5859],
         [-4.1602, -0.5859],
         [-4.1602, -0.5859]],

        [[-4.1495, -0.5742],
         [-4.0682, -0.4534],
         [-3.9991, -0.2368],
         ...,
         [-1.3176, -0.6523],
         [-1.3176, -0.6523],
         [-1.3176, -0.6523]],

        [[-1.3076, -0.6459],
         [-1.2299, -0.5946],
         [-1.1106, -0.4044],
         ...,
         [-1.2922, -0.6535],
         [-1.2922, -0.6535],
         [-1.2922, -0.6535]],

        ...,

        [[-0.9484, -0.6350],
         [-0.7619, -0.4787],
         [-0.5750, -0.2496],
         ...,
         [-1.0209, -0.6531],
         [-1.0209, -0.6531],
         [-1.0209, -0.6531]],

        [[-1.0199, -0.6580],
         [-0.9086, -0.6180],
         [-0.6951, -0.4248],
         ...,
         [-1.0291, -0.6467],
         [-1.0291, -0.6467],
         [-1.0291, -0.6467]],

        [[-1.0136, -0.6116],
       

In [ ]:

experiment_selection = 3
split_selection = 1

print("Loading ex3 runs...")
run_modules = []
run_dataloaders = []

if experiment_selection == 3:
    for e, w in get_experiment_ids(3):
        run_id = [DataSplitId(experiment=e, welding_run=w)]
        run_modules.append(ASIMoWDataModule(task="reconstruction", 
                                                batch_size=batch_size, 
                                                n_cycles=1, 
                                                val_data_ids=run_id, 
                                                test_data_ids=test_ids,
                                                shuffle_val_test=False))

    mid_idx = len(run_modules) //2
    run_mods_1 = run_modules[:mid_idx]
    run_mods_2 = run_modules[mid_idx:]
    if split_selection == 1:
        for dm in tqdm(run_mods_1, total=len(run_mods_1)):
            dm.setup(stage="test")
            run_dataloaders.append(dm.val_dataloader())
    else:
        for dm in tqdm(run_mods_2, total=len(run_mods_2)):
            dm.setup(stage="test")
            run_dataloaders.append(dm.val_dataloader())
# else:
#     for e, w in get_ex12_ids():
#         run_id = [DataSplitId(experiment=e, welding_run=w)]
#         run_modules.append(ASIMoWDataModule(task="reconstruction", 
#                                                 batch_size=batch_size, 
#                                                 n_cycles=1, 
#                                                 val_data_ids=run_id, 
#                                                 test_data_ids=test_ids,
#                                                 shuffle_val_test=False))

#     mid_idx = len(run_modules) //2
#     run_mods_1 = run_modules[:mid_idx]
#     run_mods_2 = run_modules[mid_idx:]
#     if split_selection == 1:
#         for dm in tqdm(run_mods_1, total=len(run_mods_1)):
#             dm.setup(stage="test")
#             run_dataloaders.append(dm.val_dataloader())
#     else:
#         for dm in tqdm(run_mods_2, total=len(run_mods_2)):
#             dm.setup(stage="fit")
#             run_dataloaders.append(dm.val_dataloader())


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VQVAEPatch.load_from_checkpoint(model_path)
model = model.to(device)


## Compare embedding and reconstruction losses

### Experiment 1 and 2

In [ ]:
print("#####\tExperiment 1 & 2\t#####")
embedding_losses_ex_1_2, recon_loss_ex_12, perplexity = get_losses(train_dl, model)
print_loss_stats(embedding_losses_ex_1_2, recon_loss_ex_12, perplexity)


### Experiment 3 complete

In [ ]:
print("#####\tExeriment 3\t#####")
embedding_losses_ex_3, recon_loss_ex_3, perplexity = get_losses(val_dl, model)
print_loss_stats(embedding_losses_ex_3, recon_loss_ex_3, perplexity)

### Single runs

In [ ]:
for i, dl in enumerate(run_dataloaders):
    print(f"#####\tExeriment: {experiment_selection} Split: {split_selection} Run: {i}\t#####")
    embedding_losses_ex_3_run, recon_loss_ex_3_run, perplexity = get_losses(dl, model)
    print_loss_stats(embedding_losses_ex_3_run, recon_loss_ex_3_run, perplexity)

In [ ]:
def plot_embedding_loss(dataloaders:list[tuple], model):
    emb_losses = []
    for w, dl in dataloaders:
        emb_loss, _, _ = get_losses(dl, model)
        emb_losses.append((w, emb_loss))

    plt.figure(figsize=(32, 6))
    for run in emb_losses:
        plt.plot(run[1], label=str(run[0]))
    plt.title("Embedding Loss")
    plt.legend()
    plt.show()

## Visualize Losses

In [ ]:
def plot_recon_loss(dataloaders:list[tuple], model):
    losses = []
    for w, dl in dataloaders:
        _, recon_loss, _ = get_losses(dl, model)
        losses.append((w, recon_loss))

    plt.figure(figsize=(32, 6))
    for run in losses:
        plt.plot(run[1], label=str(run[0]))
    plt.title("Reconstruction Loss")
    plt.legend()
    plt.show()

### Experiment 3

In [ ]:
# very normal: 1, 13, 15
# anomaly?: 2, 5, 14
# noteworthy: 3, 11, 12
run_nums_to_plot = [1, 13, 2, 4, 5]
dl_to_plot = []

for w, dl in enumerate(run_dataloaders):
    if w in run_nums_to_plot:
        dl_to_plot.append((w, dl))
plot_embedding_loss(dl_to_plot, model)
plot_recon_loss(dl_to_plot, model)


### Experiment 1 & 2

In [ ]:
for run in run_modules:
    del run
for dl in run_dataloaders:
    del dl
del run_modules
del run_dataloaders

## Investigating the raw data

In [ ]:
import matplotlib.pyplot as plt
patch_size = 10

In [ ]:
for batch in data_module.train_dataloader():
    data_np = batch[0].numpy()
    print(batch[0].shape)

    # Split the data into two series
    series1 = data_np[:, 0]
    series2 = data_np[:, 1]

    # Create a time axis
    time = np.arange(len(series1))

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(time, series1, label='Voltage')
    plt.plot(time, series2, label='Current')

    for x in range(0, len(series1), patch_size):
        plt.axvline(x=x, color='gray', linestyle='--', alpha=0.5)

    plt.title('Arc Welding Data')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()
    break
    

In [ ]:
def box_plot_patches(dataloader:ASIMoWDataModule, patch_size=25, cycle_length=200, run=""):
    num_patches = cycle_length // patch_size  # Calculate number of patches per cycle

    # Initialize lists to store the patch data for each time series
    patches_series1 = [[] for _ in range(num_patches)]
    patches_series2 = [[] for _ in range(num_patches)]
    # Process each batch
    for batch in dataloader:
        for j in range(batch_size):
            data_np = batch[j].numpy()

            # Assuming the data_np array is structured with two time series
            # data_np[0, :] is the first time series and data_np[1, :] is the second
            series1, series2 = data_np[:, 0], data_np[:, 1]

            # Break each series into patches and accumulate the data
            for patch_idx in range(num_patches):
                start_idx = patch_idx * patch_size
                end_idx = start_idx + patch_size
                patches_series1[patch_idx].extend(series1[start_idx:end_idx])
                patches_series2[patch_idx].extend(series2[start_idx:end_idx])

    # Convert lists to numpy arrays for boxplot
    patches_series1 = [np.array(patch) for patch in patches_series1]
    patches_series2 = [np.array(patch) for patch in patches_series2]

    # Define outlier properties
    outlier_props = dict(markerfacecolor='grey', marker='o', markersize=3, markeredgecolor='none', alpha=0.3)

    percentiles_series1 = [np.percentile(patch, [5, 95]) for patch in patches_series1]
    percentiles_series2 = [np.percentile(patch, [5, 95]) for patch in patches_series2]

    # Create boxplots
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))

    # Boxplot for the first time series
    axs[0].boxplot(patches_series1, flierprops=outlier_props, whis=[1, 99])

    axs[0].set_title('Voltage' + str(run))
    axs[0].set_xlabel('Patch')
    axs[0].set_ylabel('Value')

    # Boxplot for the second time series
    axs[1].boxplot(patches_series2, flierprops=outlier_props, whis=[1, 99])
    axs[1].set_title('Current' + str(run))
    axs[1].set_xlabel('Patch')
    axs[1].set_ylabel('Value')

    plt.tight_layout()
    plt.show()


### Experiment 1 & 2

In [ ]:
box_plot_patches(train_loader_ex1_2)

### Experiment 3 Complete

In [ ]:
box_plot_patches(train_loader_ex3)

### Experiment 3 Single runs

In [ ]:
for i, dm in enumerate(ex3_datamodules):
    box_plot_patches(dm, run=i)